<a href="https://colab.research.google.com/github/ATX24/truthDetection/blob/main/Truth_Detection_through_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Truth Detection through Machine Learning - Dhilan Shah

##Define Preliminary Functions

In [ ]:
def tokenize(text):
    clean_tokens = []
    for token in text_to_nlp(text):
        if (not token.is_stop) & (token.lemma_ != '-PRON-') & (not token.is_punct): # -PRON- is a special all inclusive "lemma" spaCy uses for any pronoun, we want to exclude these 
            clean_tokens.append(token.lemma_)
    return clean_tokens

def news(rating):
  if rating == 0:
    return 'This is real news'
  else:
    return 'This is fake news'

def tokenize_vecs(text):
    clean_tokens = []
    for token in text_to_nlp(text):
        if (not token.is_stop) & (token.lemma_ != '-PRON-') & (not token.is_punct): 
          # -PRON- is a special all inclusive "lemma" spaCy uses for any pronoun, we want to exclude these 
            clean_tokens.append(token)
    return clean_tokens



##Import Libraries

###Import NLP Data

In [ ]:
import pandas as pd   # Great for tables (google spreadsheets, microsoft excel, csv). 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk
import spacy
import wordcloud
import os # Good for navigating computer files 
import sys
pd.options.mode.chained_assignment = None #suppress warnings

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from spacy.lang.en.stop_words import STOP_WORDS
nltk.download('wordnet')
nltk.download('punkt')

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
!python -m spacy download en_core_web_md
import en_core_web_md
text_to_nlp = en_core_web_md.load()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 96.4 MB 1.2 MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-py3-none-any.whl size=98051301 sha256=479fb7ae686ced1df76bb3ad8a1b2b46b5e75903af35f2ac6d4081bb88b873e2
  Stored in directory: /tmp/pip-ephem-wheel-cache-rextxbc5/wheels/69/c5/b8/4f1c029d89238734311b3269762ab2ee325a42da2ce8edb997
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


###Import Machine Learning Data

In [ ]:
from google.colab.output import eval_js

import gdown

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score
from sklearn.base import BaseEstimator

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from keras.applications.mobilenet import MobileNet


import cv2
from google.colab import files
import requests, io, zipfile
!pip install patool
import patoolib
import os.path
from os import path
print("Downloaded Data")

     |████████████████████████████████| 56.1 MB 34 kB/s 
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


     |████████████████████████████████| 64 kB 2.1 MB/s 
     |████████████████████████████████| 77 kB 2.9 MB/s 
Downloaded Data


##Get Data

Data is uploaded from computer files (See "Datasets Used")

In [ ]:
#Only need to run once
from google.colab import files
import io

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
uploaded = files.upload()
dfTrue = pd.read_csv(io.BytesIO(uploaded['True.csv']))

In [ ]:
uploaded = files.upload()
dfFake = pd.read_csv(io.BytesIO(uploaded['Fake.csv']))

In [ ]:
real = dfTrue['text']
fake = dfFake['text']

X_text = []
y = [] #0 is true - 1 is false
tst = 0
tst2 = 0
for var in range(1150):
  ran = random.choice([0, 1])
  if ran == 0:
    X_text.append(real[var])
    y.append(0)
    tst = tst + 1
  if ran == 1:
    X_text.append(fake[var])
    y.append(1)
    tst2 = tst2 + 1

print(y)
print(X_text)
print(tst)
print(tst2)

[0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 

##Log model (Using Basic CountVectorizer)

In [ ]:
def countveclog(X_text, y, example_review):
  bow_transformer = CountVectorizer(analyzer=tokenize, max_features=1100).fit(X_text)
  x = bow_transformer.transform(X_text)
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=101)
  model = LogisticRegression()
  model.fit(X_train, y_train)
  prediction = model.predict(bow_transformer.transform([example_review]))
  return news(prediction)

##Word2vec (Used for the rest of the models)

In [ ]:
X_word2vec = []
for text in X_text:
  article = tokenize_vecs(text) # returns cleaned list of spacy tokens
  rev = [0]*300

  for word in article:
    rev += word.vector  
    
  average = rev/len(article)
  X_word2vec.append(average)
      
X_word2vec = np.array(X_word2vec)

##Different Models that Implement Word2Vec

###Log model

In [ ]:
def word2veclog( X_word2vec, y, example_review): 
  X_train, X_test, y_train, y_test = train_test_split(X_word2vec, y, test_size=0.3, random_state=101)
  model = LogisticRegression()
  model.fit(X_train, y_train)

  X_word2vecOne = []
  article = tokenize_vecs(example_review) # returns cleaned list of spacy tokens

  rev = [0]*300

  for word in article:
    rev += word.vector  
    
  average = rev/len(article)
  X_word2vecOne.append(average)

    
  X_word2vecOne = np.array(X_word2vecOne)

  prediction = model.predict(X_word2vecOne)

  y_pred = model.predict(X_train)
  acc = accuracy_score(y_train, y_pred)
  return news(prediction), acc

###KNeighborsClassifier

In [ ]:
def KNeighbors(X_word2vec, y, example_review): 
  X_train, X_test, y_train, y_test = train_test_split(X_word2vec, y, test_size=0.3, random_state=101)

  model = KNeighborsClassifier()
  model.fit(X_train, y_train)

  X_word2vecOne = []
  article = tokenize_vecs(example_review) # returns cleaned list of spacy tokens

  rev = [0]*300

  for word in article:
    rev += word.vector  
    
  average = rev/len(article)
  X_word2vecOne.append(average)

    
  X_word2vecOne = np.array(X_word2vecOne)

  prediction = model.predict(X_word2vecOne)

  y_pred = model.predict(X_train)
  acc = accuracy_score(y_train, y_pred)

  return news(prediction), acc

###GaussianProcessClassifier

In [ ]:
def GP(X_word2vec, y, example_review): 
  X_train, X_test, y_train, y_test = train_test_split(X_word2vec, y, test_size=0.3, random_state=101)

  model = GaussianProcessClassifier()
  model.fit(X_train, y_train)

  X_word2vecOne = []
  article = tokenize_vecs(example_review) # returns cleaned list of spacy tokens

  rev = [0]*300

  for word in article:
    rev += word.vector  
    
  average = rev/len(article)
  X_word2vecOne.append(average)

    
  X_word2vecOne = np.array(X_word2vecOne)
  prediction = model.predict(X_word2vecOne)

  y_pred = model.predict(X_train)
  acc = accuracy_score(y_train, y_pred)
  return news(prediction), acc

###DecisionTreeClassifier

In [ ]:
def DTree(X_word2vec, y, example_review): 
  X_train, X_test, y_train, y_test = train_test_split(X_word2vec, y, test_size=0.3, random_state=101)

  model = DecisionTreeClassifier()
  model.fit(X_train, y_train)

  X_word2vecOne = []
  article = tokenize_vecs(example_review) # returns cleaned list of spacy tokens

  rev = [0]*300

  for word in article:
    rev += word.vector  
    
  average = rev/len(article)
  X_word2vecOne.append(average)

    
  X_word2vecOne = np.array(X_word2vecOne)

  prediction = model.predict(X_word2vecOne)

  y_pred = model.predict(X_train)
  acc = accuracy_score(y_train, y_pred)
  return news(prediction), acc

###AdaBoostClassifier

In [ ]:
def Ada(X_word2vec, y, example_review): 
  X_train, X_test, y_train, y_test = train_test_split(X_word2vec, y, test_size=0.3, random_state=101)

  model = AdaBoostClassifier()
  model.fit(X_train, y_train)

  X_word2vecOne = []
  article = tokenize_vecs(example_review) # returns cleaned list of spacy tokens

  rev = [0]*300

  for word in article:
    rev += word.vector  
    
  average = rev/len(article)
  X_word2vecOne.append(average)

    
  X_word2vecOne = np.array(X_word2vecOne)

  prediction = model.predict(X_word2vecOne)

  y_pred = model.predict(X_train)
  acc = accuracy_score(y_train, y_pred)
  return news(prediction), acc

###MLProcesser

In [ ]:
def MLP(X_word2vec, y, example_review): 
  X_train, X_test, y_train, y_test = train_test_split(X_word2vec, y, test_size=0.3, random_state=101)

  model = MLPClassifier(max_iter = 1000)
  model.fit(X_train, y_train)

  X_word2vecOne = []
  article = tokenize_vecs(example_review) # returns cleaned list of spacy tokens

  rev = [0]*300

  for word in article:
    rev += word.vector  
    
  average = rev/len(article)
  X_word2vecOne.append(average)

    
  X_word2vecOne = np.array(X_word2vecOne)

  prediction = model.predict(X_word2vecOne)


  y_pred = model.predict(X_train)
  acc = accuracy_score(y_train, y_pred)
  return news(prediction), acc

###RandomForestClassifier

In [ ]:
def RFC(X_word2vec, y, example_review): 
  X_train, X_test, y_train, y_test = train_test_split(X_word2vec, y, test_size=0.3, random_state=101)

  model = RandomForestClassifier()
  model.fit(X_train, y_train)

  X_word2vecOne = []
  article = tokenize_vecs(example_review) # returns cleaned list of spacy tokens

  rev = [0]*300

  for word in article:
    rev += word.vector  
    
  average = rev/len(article)
  X_word2vecOne.append(average)

    
  X_word2vecOne = np.array(X_word2vecOne)

  prediction = model.predict(X_word2vecOne)

  y_pred = model.predict(X_train)
  acc = accuracy_score(y_train, y_pred)
  return news(prediction), acc

##Test stage

###Test from text using a multi model test function

In [ ]:
example_review = X_text[4]
y[4] #Remember 1 is fake news and 0 is real news

0

In [ ]:
example_review = input('Test: ')

Test: Experts suggest this may be the fulfillment of prophecy that foretold the glorious return of the true President who actually won the "rigged, total disaster" of an election.   "WOW! Chocolate chocolate chip!" said President Biden as the massive frozen confection was rolled across the White House lawn up to his front door. Overcome with desire, Biden dove face-first into the cone and began to try to swallow it whole like an anaconda.  With Biden distracted, Trump has taken up residence in the Oval Office and is now issuing executive orders.   While experts acknowledge this is a serious Constitutional crisis, they have so far been unable to coax Trump out from behind the Resolute Desk and have failed in pulling Biden off the ice cream cone.  Kamala Harris has been asked to run the country until authorities figure out what to do, but thus far they have been unable to remove her from the local doughnut shop. 


In [ ]:
#Uses a function that runs each model for the given data


def multiModelTest (X_word2vec, y, example_review):
  dec1 = word2veclog( X_word2vec, y, example_review)
  dec2 = KNeighbors(X_word2vec, y, example_review)
  dec3 = GP(X_word2vec, y, example_review)
  dec4 = DTree(X_word2vec, y, example_review)
  dec5 = Ada(X_word2vec, y, example_review)
  dec6 = MLP(X_word2vec, y, example_review)
  dec7 = RFC(X_word2vec, y, example_review)

  decisions = [dec1, dec2, dec3, dec4, dec5, dec5, dec7]
  print(decisions)
  fn = 0
  rn = 0
  for var in decisions:
    var = var[0]
    if var == ('This is fake news'):
      fn = fn + 1
    else:
      rn = rn + 1

  if rn > fn:
    return 'This is real news'
  else:
    return 'This is fake news'


def finalTest(X_word2vec, y, example_review):
  decA = multiModelTest(X_word2vec, y, example_review)
  decB = multiModelTest(X_word2vec, y, example_review)
  decC = multiModelTest(X_word2vec, y, example_review)

  deltaDecisions = [decA, decB, decC]
  fn1 = 0
  rn1 = 0
  for var in deltaDecisions:
    if var == ('This is fake news'):
      fn1 = fn1 + 1
    else:
      rn1 = rn1 + 1
  
  print('trial 1: ' + decA)
  print('trial 2: ' + decB)
  print('trial 3: ' + decC)

  if rn1 > fn1:
    return 'This is real news'
  else:
    return 'This is fake news' 


finalTest(X_word2vec, y, example_review)

[('This is fake news', 0.977639751552795), ('This is fake news', 0.9652173913043478), ('This is fake news', 0.968944099378882), ('This is real news', 1.0), ('This is fake news', 1.0), ('This is fake news', 1.0), ('This is fake news', 1.0)]
[('This is fake news', 0.977639751552795), ('This is fake news', 0.9652173913043478), ('This is fake news', 0.968944099378882), ('This is real news', 1.0), ('This is fake news', 1.0), ('This is fake news', 1.0), ('This is fake news', 1.0)]
[('This is fake news', 0.977639751552795), ('This is fake news', 0.9652173913043478), ('This is fake news', 0.968944099378882), ('This is real news', 1.0), ('This is fake news', 1.0), ('This is fake news', 1.0), ('This is fake news', 1.0)]
trial 1: This is fake news
trial 2: This is fake news
trial 3: This is fake news


'This is fake news'

#Datasets Used: 

https://www.uvic.ca/ecs/ece/isot/datasets/fake-news